# Various Encoder Architectures

In [7]:
import torch
import torch.nn as nn

In [8]:
## Toy Sample X,Y
SAMPLE_X = torch.rand(2, 1, 32)
SAMPLE_Y = torch.randint(2, [2])

## 0. Cute Model
    : Tutorial에서 사용했던 SimpleModel의 귀여운 버젼
    
- 여러 층의 1d Convolution block으로 encode
- 여러 층의 Fully-connected Layer로 decode

In [35]:
class CuteModel(nn.Module):
    def __init__(self):
        super(CuteModel, self).__init__()
        
        ## encoder layers : Sequence of Convolution Blocks(Conv1d + ReLU + Dropout)
        self.enc_layer1 = nn.Conv1d(1, 8, kernel_size=5, stride=2)
        self.enc_layer2 = nn.Conv1d(8, 8, kernel_size=5, stride=2)
        
        ## classifier layers : Multi-Layer Perceptron(FC + ReLU)
        self.decoder = nn.Linear(8, 6)
        
    def forward(self, x):
        print('Input size :\t\t',x.size())
        
        x = self.enc_layer1(x)
        print('After 1st layer :\t',x.size())
        x = self.enc_layer2(x)
        print('After 2nd layer :\t',x.size())
        
        x = x.mean(dim=-1)
        print('After mean pool :\t',x.size())
        
        x = self.decoder(x)
        print('After decoder :\t\t',x.size())
        return x

In [36]:
model = CuteModel()
p = model(SAMPLE_X)
loss = nn.CrossEntropyLoss()(p, SAMPLE_Y)
print('[loss] : ', loss)

Input size :		 torch.Size([2, 1, 32])
After 1st layer :	 torch.Size([2, 8, 14])
After 2nd layer :	 torch.Size([2, 8, 5])
After mean pool :	 torch.Size([2, 8])
After decoder :		 torch.Size([2, 6])
[loss] :  tensor(1.8711, grad_fn=<NllLossBackward>)


## 1. DilatedCNN
< WaveNet >
<img src="../resources/WaveNet.png" width="500">

- dilated CNN을 통해 input의 주기적 특성을 파악할 수 있습니다.
- 각 *self.enc_layer#*을 정의할 때, nn.Conv1d의 인자로 dilation을 정의할 수 있습니다. (기본 : 1)

(ex)
<pre>
self.enc_layer1 = nn.Conv1d(1, 8, kernel_size=5, stride=3, <b>dilation=2</b>)
</pre>


In [37]:
class DilatedCNNModel(nn.Module):
    def __init__(self):
        super(DilatedCNNModel, self).__init__()
        
        ## encoder layers : Sequence of Convolution Blocks(Conv1d + ReLU + Dropout)
        ## you can define dilation here
        self.enc_layer1 = nn.Conv1d(1, 8, kernel_size=5, stride=2, dilation=2)
        self.enc_layer2 = nn.Conv1d(8, 8, kernel_size=5, stride=2, dilation=2)
        
        ## classifier layers : Multi-Layer Perceptron(FC + ReLU)
        self.decoder = nn.Linear(8, 6)
        
    def forward(self, x):
        print('Input size :\t\t',x.size())

        x = self.enc_layer1(x)
        print('After 1st layer :\t',x.size())
        x = self.enc_layer2(x)
        print('After 2nd layer :\t',x.size())
        
        x = x.mean(dim=-1)
        print('After mean pool :\t',x.size())
        
        x = self.decoder(x)
        print('After decoder :\t\t',x.size())
        return x

In [38]:
model = DilatedCNNModel()
p = model(SAMPLE_X)
loss = nn.CrossEntropyLoss()(p, SAMPLE_Y)
print('[loss] : ', loss)

Input size :		 torch.Size([2, 1, 32])
After 1st layer :	 torch.Size([2, 8, 12])
After 2nd layer :	 torch.Size([2, 8, 2])
After mean pool :	 torch.Size([2, 8])
After decoder :		 torch.Size([2, 6])
[loss] :  tensor(1.6990, grad_fn=<NllLossBackward>)


## 2. Cumulate each layer outputs

<img src='../resources/cumulate.png' width='600'>

- 각 인코딩 레이어의 output을 concatenate함으로써, 고주파 특성과 저주파 특성을 동시에 반영할 수 있습니다.

(ex)
<pre>
x = torch.cat([x1,x2], dim=-1)
</pre>

In [42]:
class CumulateModel(nn.Module):
    def __init__(self):
        super(CumulateModel, self).__init__()
        
        ## encoder layers : Sequence of Convolution Blocks(Conv1d + ReLU + Dropout)
        self.enc_layer1 = nn.Conv1d(1, 8, kernel_size=5, stride=3)
        self.enc_layer2 = nn.Conv1d(8, 8, kernel_size=5, stride=3)
        
        ## classifier layers : Multi-Layer Perceptron(FC + ReLU)
        self.decoder = nn.Linear(2*8, 6)
        
    def forward(self, x):
        print('Input size :\t\t',x.size())

        x1 = self.enc_layer1(x)
        print('After 1st layer :\t',x1.size())
        x2 = self.enc_layer2(x1)
        print('After 2nd layer :\t',x2.size())
        
        ## average pool each layer outputs
        x1 = x1.mean(dim=-1)
        print('After pool x1 :\t\t',x1.size())
        x2 = x2.mean(dim=-1)
        print('After pool x2 :\t\t',x2.size())
        
        ## concatenate
        x = torch.cat([x1,x2], dim=-1)
        print('After concat :\t\t', x.size())
        
        x = self.decoder(x)
        print('After decoder :\t\t',x2.size())
        return x

In [43]:
model = CumulateModel()
p = model(SAMPLE_X)
loss = nn.CrossEntropyLoss()(p, SAMPLE_Y)
print('[loss] : ', loss)

Input size :		 torch.Size([2, 1, 32])
After 1st layer :	 torch.Size([2, 8, 10])
After 2nd layer :	 torch.Size([2, 8, 2])
After pool x1 :		 torch.Size([2, 8])
After pool x2 :		 torch.Size([2, 8])
After concat :		 torch.Size([2, 16])
After decoder :		 torch.Size([2, 8])
[loss] :  tensor(1.9182, grad_fn=<NllLossBackward>)


## 3. Variational Model

<img src='../resources/variational.png' width='600'>

- *z* 분포의 **mean** 과 **log variance**를 계산할 수 있습니다.
<pre>
z_mu = self.enc_mu(x)
z_logvar = self.enc_logvar(x)
</pre>

- *z_mu*와 *z_logvar*로부터, **reparameterization trick**을 통해 *z*를 샘플링 할 수 있습니다.
<pre>
z = self._reparam(z_mu, z_logvar)
</pre>

- z 분포를 Prior 분포인 N(0,1)과 유사하도록 학습하기 위해, **KL-Divergence**를 낮추도록 학습합니다.
<pre>
self.kld = self._kld_gauss(z_mu, z_logvar)
</pre>

- **reparameterization trick**과 **KL-Divergence** 계산을 위한 함수는 걱정말고 샘플코드를 가져다 쓰세요!
<pre>
def _reparam(self, mu, logvar):
def _kld_gauss(self, mu, logvar):
</pre>


In [48]:
class VariationalModel(nn.Module):
    def __init__(self):
        super(VariationalModel, self).__init__()
        
        ## encoder layers : Sequence of Convolution Blocks(Conv1d + ReLU + Dropout)
        self.enc_layer1 = nn.Conv1d(1, 8, kernel_size=5, stride=3)
        self.enc_layer2 = nn.Conv1d(8, 8, kernel_size=5, stride=3)
        
        self.enc_mu = nn.Linear(8, 8)
        self.enc_logvar = nn.Linear(8, 8)
        
        self.kld = torch.tensor(0)
        
        ## classifier layers : Multi-Layer Perceptron(FC + ReLU)
        self.decoder = nn.Linear(8, 6)
        
    def forward(self, x):
        print('Input size :\t\t',x.size())
        
        x = self.enc_layer1(x)
        print('After 1st layer :\t',x.size())
        x = self.enc_layer2(x)
        print('After 2nd layer :\t',x.size())
        
        x = x.mean(dim=-1)
        print('After mean pool :\t',x.size())
        
        z_mu = self.enc_mu(x)
        z_logvar = self.enc_logvar(x)
        print('z_mu & z_logvar :\t',z_mu.size())
        z = self._reparam(z_mu, z_logvar)
        print('Sampled z shape :\t',z.size())
        
        self.kld = self._kld_gauss(z_mu, z_logvar)
        
        x = self.decoder(z)
        print('After decoder :\t\t',x.size())
        return x
    
    def _reparam(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = torch.autograd.Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def _kld_gauss(self, mu, logvar):
        kld_element = -logvar + torch.exp(logvar) + mu**2 - 1
        return 0.5 * torch.mean(kld_element)

***(IMPORTANT!!!)*** variational model을 사용할 때, latent distribution을 prior에 근사시키기 위해서는 loss term에 ***KL-Divergence term***을 추가해야 합니다.

<pre>
...
class_loss = CrossEntropyLoss()(pred, target)
<b>kld_loss = model.kld</b>
total_loss = class_loss + <b>BETA*kld_loss</b>
total_loss.backward()
...
</pre>

또한, BETA를 통해 각 loss term의 비중을 조절할 수 있습니다. (ex: BETA=0.1 or BETA=10)

In [49]:
BETA = 1
model = VariationalModel()
p = model(SAMPLE_X)
loss = nn.CrossEntropyLoss()(p, SAMPLE_Y) + BETA*model.kld
print('[loss] : ', loss)

Input size :		 torch.Size([2, 1, 32])
After 1st layer :	 torch.Size([2, 8, 10])
After 2nd layer :	 torch.Size([2, 8, 2])
After mean pool :	 torch.Size([2, 8])
z_mu & z_logvar :	 torch.Size([2, 8])
Sampled z shape :	 torch.Size([2, 8])
After decoder :		 torch.Size([2, 6])
[loss] :  tensor(1.6636, grad_fn=<AddBackward0>)


## Try build your own model architecture!!